In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

DATA_PATH = "../donnees/ecommerce/"

In [ ]:
# Chargement du parquet F_ACHATS créé dans l'analyse 1
F_ACHATS = pd.read_parquet("f_achats.parquet")

print("F_ACHATS chargé depuis f_achats.parquet")
print("Shape :", F_ACHATS.shape)
print("Colonnes :", list(F_ACHATS.columns))
display(F_ACHATS.head(3))

In [ ]:
# Recréation des dimensions de temps (locales, non sauvegardées dans le parquet)
F_ACHATS["year"]  = F_ACHATS["purchase_timestamp"].dt.year
F_ACHATS["month"] = F_ACHATS["purchase_timestamp"].dt.month
F_ACHATS["date"]  = F_ACHATS["purchase_timestamp"].dt.normalize()
F_ACHATS["week"]   = F_ACHATS["purchase_timestamp"].dt.isocalendar().week
F_ACHATS["weekday"] = F_ACHATS["purchase_timestamp"].dt.weekday
F_ACHATS["hour"]    = F_ACHATS["purchase_timestamp"].dt.hour

print("Colonnes temps ajoutées : year, month, date, week, weekday, hour")
display(
    F_ACHATS[["purchase_timestamp", "year", "month", "date", "week", "weekday", "hour"]]
    .head(3)
)

In [ ]:
# Chargement des dimensions nécessaires pour l'analyse
products  = pd.read_parquet(DATA_PATH + "products.parquet")
customers = pd.read_parquet(DATA_PATH + "customers.parquet")

print("Dimensions chargées :")
print("products :", products.shape)
print("customers:", customers.shape)

# Préparation de la dimension produit
prod_dim = products[["product_id", "category_name"]].copy()

# Préparation de la dimension client avec macro-région
cust_dim = customers[["customer_id", "state", "name_state"]].copy()

region_map = {
    "SP": "Sudeste", "RJ": "Sudeste", "MG": "Sudeste", "ES": "Sudeste",
    "RS": "Sul", "SC": "Sul", "PR": "Sul",
    "BA": "Nordeste", "PE": "Nordeste", "CE": "Nordeste", "RN": "Nordeste",
    "PB": "Nordeste", "AL": "Nordeste", "SE": "Nordeste", "PI": "Nordeste",
    "MA": "Nordeste",
    "DF": "Centro-Oeste", "GO": "Centro-Oeste", "MT": "Centro-Oeste", "MS": "Centro-Oeste",
    "AM": "Norte", "PA": "Norte", "RO": "Norte", "RR": "Norte",
    "AP": "Norte", "TO": "Norte", "AC": "Norte"
}

cust_dim["macro_region"] = cust_dim["state"].map(region_map)

print("Dimensions produit et client préparées")

In [ ]:
# Analyse 2 : Structure des paniers & pression logistique sur les segments clés
# Dans cette deuxième analyse sur l’étoile 2, on part des segments dominants
# identifiés dans l’analyse 1 (catégories phares et régions clés) et on regarde
# plus finement la structure des paniers : niveau de line_total, part du
# freight_value et contraintes de délai (limit). L’idée est de voir quels couples
# produit × région combinent à la fois fort volume, paniers élevés et poids
# logistique important, car ce sont eux qui risquent de poser le plus de problèmes
# opérationnels quand la demande explose.


In [ ]:
# Hypothèse 2
# Puisqu’on a montré que le line_total est très concentré sur quelques catégories
# et sur la macro-région Sudeste, on suppose maintenant que ces mêmes segments
# présentent aussi les paniers les plus élevés et les coûts logistiques les plus forts.
# Autrement dit, les couples produit × région qui tirent le business sont aussi ceux
# où la pression logistique est maximale, ce qui en fait des zones critiques en cas
# de saturation et explique une partie des insatisfactions observées dans l’étoile 1.


In [ ]:
# 1) Évolution annuelle du panier moyen et de la part logistique (line_total_mean vs freight_share)
# 2) Structure globale du panier : panier moyen et part logistique par catégorie de produit (Top 10)
# 3) Structure du panier et part logistique par macro-région client (Sudeste, Sul, Nordeste, etc.)
# 4) Heatmap panier moyen × part logistique par couple catégorie de produit × macro-région
# 5) Délai moyen limit (h) par catégorie de produit (Top 10) – pression temporelle par segment
# 6) Délai moyen limit (h) par macro-région et par état client – contraintes logistiques géographiques
# 7) Segments critiques : panier moyen, part logistique et délai moyen par couple catégorie × macro-région (Top 10)
# 8) Mois de pic : évolution quotidienne du panier moyen et de la part logistique (tous clients)
# 9) Mois de pic : comportement logistique des segments critiques (catégorie × macro-région) au jour le jour